<a href="https://www.kaggle.com/code/averma111/store-sales-multiple-models?scriptVersionId=125858296" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [2]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import warnings
warnings.filterwarnings('ignore')

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/store-sales-time-series-forecasting/oil.csv
/kaggle/input/store-sales-time-series-forecasting/sample_submission.csv
/kaggle/input/store-sales-time-series-forecasting/holidays_events.csv
/kaggle/input/store-sales-time-series-forecasting/stores.csv
/kaggle/input/store-sales-time-series-forecasting/train.csv
/kaggle/input/store-sales-time-series-forecasting/test.csv
/kaggle/input/store-sales-time-series-forecasting/transactions.csv


In [3]:
ROOT_PATH = '/kaggle/input/store-sales-time-series-forecasting'
oil = pd.read_csv(ROOT_PATH+'/oil.csv')
stores = pd.read_csv(ROOT_PATH+'/stores.csv')
train = pd.read_csv(ROOT_PATH+'/train.csv')
test = pd.read_csv(ROOT_PATH+'/test.csv')
transactions = pd.read_csv(ROOT_PATH+'/transactions.csv')

In [4]:
## https://www.kaggle.com/code/arjanso/reducing-dataframe-memory-size-by-65
def reduce_memory_usage(df):
    
    start_mem = df.memory_usage().sum() / 1024 ** 2
    print('Memory usage of dataframe is {:.2f} MB'.format(start_mem))
    
    for col in df.columns:
        col_type = df[col].dtype.name
        if ((col_type != 'datetime64[ns]') & (col_type != 'category')):
            if (col_type != 'object'):
                c_min = df[col].min()
                c_max = df[col].max()

                if str(col_type)[:3] == 'int':
                    if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                        df[col] = df[col].astype(np.int8)
                    elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                        df[col] = df[col].astype(np.int16)
                    elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                        df[col] = df[col].astype(np.int32)
                    elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                        df[col] = df[col].astype(np.int64)

                else:
                    if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                        df[col] = df[col].astype(np.float16)
                    elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                        df[col] = df[col].astype(np.float32)
                    else:
                        pass
            else:
                df[col] = df[col].astype('category')
    mem_usg = df.memory_usage().sum() / 1024 ** 2 
    print("Memory usage became: ",mem_usg," MB")
    
    return df

In [5]:
train = reduce_memory_usage(train)
test = reduce_memory_usage(test)

Memory usage of dataframe is 137.37 MB
Memory usage became:  40.14344787597656  MB
Memory usage of dataframe is 1.09 MB
Memory usage became:  0.24676895141601562  MB


In [6]:
def summary(text, df):
    print(f'{text} shape: {df.shape}')
    summ = pd.DataFrame(df.dtypes, columns=['dtypes'])
    summ['null'] = df.isnull().sum()
    summ['unique'] = df.nunique()
    summ['min'] = df.min()
    summ['median'] = df.median()
    summ['max'] = df.max()
    summ['mean'] = df.mean()
    summ['std'] = df.std()
    return summ

In [7]:
summary('oil',oil)

oil shape: (1218, 2)


,dtypes,null,unique,min,median,max,mean,std
date,object,0,1218,2013-01-01,NaN,2017-08-31,NaN,NaN
dcoilwtico,float64,43,998,26.19,53.19,110.62,67.714366,25.630476


In [8]:
summary('stores',stores)

stores shape: (54, 5)


,dtypes,null,unique,min,median,max,mean,std
store_nbr,int64,0,54,1,27.5,54,27.500000,15.732133
city,object,0,22,Ambato,NaN,Santo Domingo,NaN,NaN
state,object,0,16,Azuay,NaN,Tungurahua,NaN,NaN
type,object,0,5,A,NaN,E,NaN,NaN
cluster,int64,0,17,1,8.5,17,8.481481,4.693395


In [9]:
summary('train',train)

train shape: (3000888, 6)


,dtypes,null,unique,min,median,max,mean,std
id,int32,0,3000888,0.0,1500443.5,3000887.0,1.500444e+06,866281.891642
date,category,0,1684,NaN,NaN,NaN,NaN,NaN
store_nbr,int8,0,54,1.0,27.5,54.0,2.750000e+01,15.585787
family,category,0,33,NaN,NaN,NaN,NaN,NaN
sales,float32,0,379610,0.0,11.0,124717.0,3.577758e+02,1101.997681
onpromotion,int16,0,362,0.0,0.0,741.0,2.602770e+00,12.218882


In [10]:
summary('test',test)

test shape: (28512, 5)


,dtypes,null,unique,min,median,max,mean,std
id,int32,0,28512,3000888.0,3015143.5,3029399.0,3.015144e+06,8230.849774
date,category,0,16,NaN,NaN,NaN,NaN,NaN
store_nbr,int8,0,54,1.0,27.5,54.0,2.750000e+01,15.586057
family,category,0,33,NaN,NaN,NaN,NaN,NaN
onpromotion,int16,0,212,0.0,0.0,646.0,6.965383e+00,20.683952


In [11]:
summary('transactions',transactions)

transactions shape: (83488, 3)


,dtypes,null,unique,min,median,max,mean,std
date,object,0,1682,2013-01-01,NaN,2017-08-15,NaN,NaN
store_nbr,int64,0,54,1,27.0,54,26.939237,15.608204
transactions,int64,0,4993,5,1393.0,8359,1694.602158,963.286644


In [20]:
stores_transactions = pd.merge(stores, transactions, on=["store_nbr"])


In [21]:
summary('stores_transactions',stores_transactions)

stores_transactions shape: (83488, 7)


,dtypes,null,unique,min,median,max,mean,std
store_nbr,int64,0,54,1,27.0,54,26.939237,15.608204
city,object,0,22,Ambato,NaN,Santo Domingo,NaN,NaN
state,object,0,16,Azuay,NaN,Tungurahua,NaN,NaN
type,object,0,5,A,NaN,E,NaN,NaN
cluster,int64,0,17,1,9.0,17,8.533705,4.724351
date,object,0,1682,2013-01-01,NaN,2017-08-15,NaN,NaN
transactions,int64,0,4993,5,1393.0,8359,1694.602158,963.286644


In [22]:
stores_transactions_oil = pd.merge(stores_transactions, oil, on=["date"])

In [24]:
summary('stores_transactions_oil',stores_transactions_oil)

stores_transactions_oil shape: (59651, 8)


,dtypes,null,unique,min,median,max,mean,std
store_nbr,int64,0,54,1,27.00,54,26.936397,15.608561
city,object,0,22,Ambato,NaN,Santo Domingo,NaN,NaN
state,object,0,16,Azuay,NaN,Tungurahua,NaN,NaN
type,object,0,5,A,NaN,E,NaN,NaN
cluster,int64,0,17,1,9.00,17,8.533822,4.724132
date,object,0,1202,2013-01-01,NaN,2017-08-15,NaN,NaN
transactions,int64,0,4273,5,1357.00,8359,1612.411896,869.673648
dcoilwtico,float64,1803,994,26.19,52.66,110.62,66.638287,25.466567


In [25]:
df_full = pd.merge(train, stores_transactions_oil, how="left", on=["store_nbr","date"])

In [ ]:
summary('df_full',df_full)

df_full shape: (1968483, 12)
